In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager as CM
from datetime import datetime, timedelta
import time
import sys
import warnings
import configparser
warnings.filterwarnings("ignore")

import sqlite3 as db
import numpy as np
import pandas as pd

In [26]:
class FantasyScraper:
    '''
    
    
    '''

    def __init__(self, 
    headless=False,
    # keep incognito mode off, or else code will not work (TODO: fix this)
    incognito=False, 
    url='https://www.laligafantasymarca.com/',
    filename='login.txt'):
        '''
        
        
        '''

        self.headless = headless
        self.incognito = incognito
        self.url = url
        self.filename = filename
    
    def get_login_from_config(self):
        '''
        
        
        '''

        config = configparser.ConfigParser()
        print("[OK] Reading default config file...")
        try:
            config.read(self.filename)
        except:
            raise Exception(f"[ERROR] Config file not found. Please check {self.filename} in your current directory.")
        
        try:
            username = config.get("login", "mail")
            password = config.get("login", "password")
            print("[OK] Config file read successfully.")
        except:
            raise Exception("Email and password were not found / are not readable. Check {self.filename} in your current directory.")

        self.username = username
        self.password = password

    def update_config(self):
        '''
        
        
        '''

        # Ask the user for email and password
        username = input("Enter your mail: ")
        password = input("Enter your password: ")

        # the user can also save the new credentials to the config file (overwrite the old one)
        remember = input("Do you want to remember your password? (y/n) ")
        if remember == "y":
            config = configparser.ConfigParser()
            config.read(self.filename)
            config.set("login", "mail", username)
            config.set("login", "password", password)

            with open(self.filename, "w") as configfile:
                config.write(configfile)
        if remember == "n":
            pass
        if remember != "y" and remember != "n":
            self.update_config()

        self.username = username
        self.password = password


    # TO-DO: Make only valid inputs 1-0 with a while loop

    def config(self):
        '''
        
        
        '''

        # let the user choose if they want to use the config file or not
        choice = input("Do you want to use the default config file? (y/n/exit)")

        # if the user wants to use the config file, read it
        if choice == "y":
            self.get_login_from_config()

        # if the user doesn't want to use the config file, ask for the credentials
        elif choice == "n":
            self.update_config()

        # if the user wants to exit the program, exit
        elif choice == "exit":
            sys.exit('Exiting...')

        # if the user doesn't enter a valid input, ask again
        else:
            self.config()
            

    def create_driver(self):
        '''
        
        
        '''

        # Set browser options
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument("--log-level=3")
        options.add_argument("--silent")
        # Makes the browser run in background
        if self.headless:
            options.add_argument("--headless")
        # Makes the browser run in incognito mode
        if self.incognito:
            options.add_argument("--incognito")
        # Generate the driver
        browser = webdriver.Chrome(executable_path=CM().install(), options=options)
        print('[OK] Browser created')
        time.sleep(1)
        # Set window size to full screen
        browser.set_window_size(1920, 1080)
        print('[OK] Window size set to 1920x1080')
        time.sleep(2)
        self.browser = browser

    def open_page(self):
        '''
        
        
        '''

        # Open the Website
        try:
            self.browser.get(self.url)
            print('[OK] Page opened succesfully')
        except:
            raise Exception('[ERROR] Page cannot be opened. Check your internet connection')

    def reject_cookies(self):
        '''
        
        
        '''

        print('[OK] [STEP 1] Rejecting Cookies...')
        time.sleep(3.5)
        # Access the login page step 1
        try:
            self.browser.find_element(By.CLASS_NAME,"cookie-setting-link").click()
            print('[OK] [STEP 1] Accesing Manage Preferences page')
        except:
            raise Exception('[ERROR] [STEP 1] Manage Preferences page cannot be accessed. Check your internet connection')
            
        time.sleep(1)
        try:
            self.browser.find_element(By.CLASS_NAME,"ot-pc-refuse-all-handler").click()
            print('[OK] [STEP 1] Refusing all cookies')
        except:
            raise Exception('[ERROR] [STEP 1] Something happened while refusing all cookies. Check your internet connection')
            
    def login_2(self):
        '''
        
        
        '''

        print('[OK] [STEP 2] Accessing the page...')
        time.sleep(2)
        # Access the login page step 2
        try:
            self.browser.find_element(By.CLASS_NAME,"btn-fantasy.grey.big.btn-secondary").click()
            print('[OK] [STEP 2] Login page accessed succesfully')
        except:
            raise Exception('[ERROR] [STEP 2] Login page cannot be accessed. Check your internet connection')
        
    def input_credentials(self):
        '''
        
        
        '''

        time.sleep(2)
        # Fill credentials
        self.browser.find_element(By.NAME, 'username').send_keys(self.username)
        print('[OK] [STEP 3] Email entered succesfully')
        time.sleep(1)
        self.browser.find_element(By.NAME, 'password_current').send_keys(self.password)
        print('[OK] [STEP 3] Password entered succesfully')
        time.sleep(1.5)
        # Click Log In
        print('[OK] [STEP 3] Clicking Log In...')
        try:
            self.browser.find_element(By.CLASS_NAME,'btn-fantasy.green.big').click()
            time.sleep(3.5)
        except:
            raise Exception('[ERROR] [STEP 3] Log In button cannot be clicked. Check your internet connection')

    def access_league(self):
        '''
        
        
        '''

        time.sleep(7)
        print('[OK] [STEP 4] Accessing the league...')
        # Accessing the league page
        try:
            self.browser.find_element(By.CLASS_NAME,"ltr.team").click()
            print('[OK] [STEP 4] League page accessed succesfully')
        except:
            raise Exception('[ERROR] [STEP 4] League page cannot be accessed. Check your internet connection')

    def access_market(self):
        '''
        
        
        '''

        time.sleep(7)
            
        print('[OK] [STEP 5] Accessing the Market to get the info...')
        try:
            self.browser.find_element(By.XPATH,"/html/body/fy-app/fy-layout-game/div/div/div/fy-sidenav/aside/nav/div/ul/li[3]").click()
            print('[OK] [STEP 5] Market page accessed succesfully')
        except:
            raise Exception('[ERROR] [STEP 5] Market page cannot be accessed. Check your internet connection')

        self.remove_market_warning()
        # Switching to the search bar 
        time.sleep(1)
        try:
            self.browser.find_element(By.ID,"searchPlayer-tab").click()
            print('[OK] [STEP 5] succesfully accessed to search menu')
        except:
            raise Exception('[ERROR] [STEP 5] Market page cannot be accessed. Check your internet connection')
    
    def remove_market_warning(self):
        '''
        
        
        '''

        try:
                self.browser.find_element(
                    by=By.XPATH,
                    value="/html/body/modal-container/div/div/fy-warning-modal/div[2]/button"
                ).click()
        except:
                print('[OK] [STEP 5] No warning message')

    def show_all_players(self):
        '''
        
        
        '''

        time.sleep(1)
        print('[OK] [STEP 5] Showing All Players')
        try:
            self.browser.find_element(By.XPATH,"/html/body/fy-app/fy-layout-game/div/div/div/main/fy-league-market/div/div/fy-tabs-container/div/div[4]/div/div[1]/div/fy-search-player/div/div[4]/div/div/button").click()
            print('[OK] [STEP 5] Open dropdown')
        except:
            raise Exception('[ERROR] [STEP 5] Something went wrong opening the dropdown menu')
        try:
            self.browser.find_element(By.XPATH,"/html/body/fy-app/fy-layout-game/div/div/div/main/fy-league-market/div/div/fy-tabs-container/div/div[4]/div/div[1]/div/fy-search-player/div/div[4]/div/div/div/a[1]").click()
            print('[OK] [STEP 5] All players now aviable')
        except:
            raise Exception('[ERROR] [STEP 5] Something went wrong while selecting inside the dropdown menu')

    def get_players_df(self):
        '''
        
        
        '''

        players = []
        for player in self.browser.find_elements(By.CLASS_NAME,"player-item"):
            text = player.text.replace('PFSY','\n').split(sep='\n')
            text  = list(map(lambda x: x.strip(), text))
            players.append(text)
        df = pd.DataFrame(players, columns= ['Name','Position','Owner','Price', 'Team', 'PSFY_current', 'PSFY_last_year'])
        df['Date_actua'] = pd.to_datetime("today").strftime("%d/%m/%Y")
        df['Price'] = df['Price'].str.replace('.','').astype('int')
        self.df = df

    def save_in_db(self):
        '''
        
        
        '''

        conn = db.connect('../data/daily_price.db')
        self.df.to_sql("players_price", conn, if_exists="append")
        conn.close()

    def scrape(self):
        '''
        
        
        '''
        self.config()
        self.create_driver()
        self.open_page()
        self.reject_cookies()
        self.login_2()
        self.input_credentials()
        self.access_league()
        self.access_market()
        self.show_all_players()
        self.get_players_df()
        self.save_in_db()

In [27]:
FantasyScraper(headless=False).scrape()

[OK] Reading default config file...
[OK] Config file read successfully.



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\Usuario\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


[OK] Browser created
[OK] Window size set to 1920x1080
[OK] Page opened succesfully
[OK] [STEP 1] Rejecting Cookies...
[OK] [STEP 1] Accesing Manage Preferences page
[OK] [STEP 1] Refusing all cookies
[OK] [STEP 2] Accessing the page...
[OK] [STEP 2] Login page accessed succesfully
[OK] [STEP 3] Email entered succesfully
[OK] [STEP 3] Password entered succesfully
[OK] [STEP 3] Clicking Log In...
[OK] [STEP 4] Accessing the league...
[OK] [STEP 4] League page accessed succesfully
[OK] [STEP 5] Accessing the Market to get the info...
[OK] [STEP 5] Market page accessed succesfully
[OK] [STEP 5] succesfully accessed to search menu
[OK] [STEP 5] Showing All Players
[OK] [STEP 5] Open dropdown
[OK] [STEP 5] All players now aviable


OperationalError: unable to open database file